In [42]:
import bs4
import pandas as pd
import tkinter as tk
from tkinter import ttk
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def show_content():
    print('查詢完成')
    
    #取得父級的空格內容
    C1 = city_dp1.get()
    C2 = city_dp2.get()
    
    new_window = tk.Toplevel() #產生新表單的關鍵 Toplevel()
    #在小表單中建立兩個欄位顯示
    new_window.title(C1 + '至' + C2 + '高鐵時刻表')
    new_window.geometry('300x300')
    
    #文字顯示
    L1 = tk.Label(new_window, text='啟程站為: '+ C1,  fg='blue', font='Arial 15', width='15')
    L1.grid(column=0, row=0)
    L2 = tk.Label(new_window, text='到達站為: '+ C2,  fg='blue', font='Arial 15', width='15')
    L2.grid(column=0, row=1)
    
    #使用selenium傳入值
    url = 'https://www.thsrc.com.tw/'
    driver = webdriver.Chrome('chromedriver')
    driver.get(url)
    driver.maximize_window()
    
    #我同意按鈕
    driver.find_element(By.CLASS_NAME,'swal2-confirm').click() 
    
    #啟程站選單
    stations = driver.find_element(By.ID, 'select_location01') #找到起程站id並且使用for迴圈列出option所有選項
    stas = stations.find_elements(By.TAG_NAME, 'option')
    for s in stas:
        if s.text == C1:
            s.click()
    time.sleep(3)   

    #到達站選單
    endstations = driver.find_element(By.ID, 'select_location02') #找到到達站id並且使用for迴圈列出option所有選項
    endstas = endstations.find_elements(By.TAG_NAME, 'option')
    for e in endstas:
        if e.text == C2:
            e.click()
    time.sleep(3) 
    
    #查詢按鈕
    btn = driver.find_element(By.ID, 'start-search')
    btn.click()
    time.sleep(10)
    
    #使用Beautifulsoup
    soup = bs4.BeautifulSoup(driver.page_source,'lxml')

    #車次
    tickets = soup.find_all('div', class_='tr-td train')

    #出發時間與到達時間
    times = soup.find_all('span', class_='darkgray')

    #組成list
    tlist = []
    i=0
    for t in tickets:
        nlist = []
        nlist.append(t.text.strip())
        nlist.append(times[i].text.strip())
        nlist.append(times[i+1].text.strip())
        tlist.append(nlist)
        i += 2
        
    
    #轉成DataFrame
    df = pd.DataFrame(tlist, columns=["車次","出發時間","到達時間"])
    df.index += 1
    
    #=========表格框線
    style = ttk.Style()
    style.theme_use('default')
    style.configure("Treeview", font=('Arial',12), rowheight=40, borderwidth=2)
    style.configure("Treeview.Heading", font=('Arial',12,'bold'), borderwidth=2)

    tree = ttk.Treeview(new_window,style="Treeview") #定義表格物件
    tree['columns'] = list(df.columns) #定義表單的欄位等於pandas的欄位
    tree.column("#0", width=0, stretch=tk.NO) #tk的欄位設定

    for col in tree['columns']:
        tree.column(col,width=100)
        tree.heading(col,text=col)

    #=====把資料填入 tk中 遍歷iterrows
    for i,row in df.iterrows():
        values = list(row)
        tree.insert("", "end", text=i, values=values) #四個參數
    tree.grid(column=0, row=2)

    
    
#==========主視窗(選擇頁面)
root = tk.Tk()
root.title('高鐵時刻查詢系統')
root.geometry('450x200')

#===========空格欄位
space = tk.Label(root,text='', font='Arial 15', width='20') 
space.grid(column=0, row=0)

#===========欄位
start = tk.Label(root,text='啟程站', font='Arial 15', width='15') 
start.grid(column=0, row=1)

end = tk.Label(root,text='到達站', font='Arial 15', width='15') 
end.grid(column=0, row=2)

#===========下拉選單
cities = ['請選擇', '南港', '台北', '板橋', '桃園', '新竹', '苗栗', '台中', '彰化', '雲林', '嘉義', '台南', '高雄']
sel_city = tk.StringVar() #建立文字串列的類型，來儲存選中的城市名稱
city_dp1 = ttk.Combobox(root, textvariable=sel_city, values=cities, font='Arial 15', width='15')

cities = ['請選擇', '南港', '台北', '板橋', '桃園', '新竹', '苗栗', '台中', '彰化', '雲林', '嘉義', '台南', '高雄']
sel_city = tk.StringVar() #建立文字串列的類型，來儲存選中的城市名稱
city_dp2 = ttk.Combobox(root, textvariable=sel_city, values=cities, font='Arial 15', width='15')

city_dp1.current(0)
city_dp1.grid(column=1, row=1)

city_dp2.current(0)
city_dp2.grid(column=1, row=2)

#===========按鈕
btn1 = tk.Button(root, text='查詢', font='Arial 15', command=show_content)
btn2 = tk.Button(root, text='重設', font='Arial 15')
btn1.grid(column=0, row=4)
btn2.grid(column=1, row=4)
#===========空格欄位
space = tk.Label(root,text='', font='Arial 15', width='20') 
space.grid(column=0, row=3)

root.mainloop() 

查詢完成
